In [8]:
import sys
from os.path import join
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from tensorflow.python.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50

import pandas as pd
import numpy as np
#import keras
#from keras import backend as K


import tensorflow 
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [2]:
train_dir ='F:/yasmin/roi/train_res'
test_dir = 'F:/yasmin/roi/test_res'

In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.resnet50.preprocess_input,
    rotation_range=25.,
    channel_shift_range=20.,
    validation_split=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
        )

test_datagen = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.resnet50.preprocess_input
    )


In [4]:
train_batches = train_datagen.flow_from_directory(train_dir, target_size=(224,224),
    classes=['benign_calcification','benign_mass','malignant_mass','malignant_calcification','nothing'],color_mode='rgb', batch_size=64,subset='training')
validation_batches = train_datagen.flow_from_directory(train_dir, target_size=(224,224),
    classes=['benign_calcification','benign_mass','malignant_mass','malignant_calcification','nothing'],color_mode='rgb', batch_size=64,subset='validation')
test_batches = test_datagen.flow_from_directory(test_dir, target_size=(224,224),
    classes=['benign_calcification','benign_mass','malignant_mass','malignant_calcification','nothing'],color_mode='rgb', batch_size=1)


Found 5052 images belonging to 5 classes.
Found 1262 images belonging to 5 classes.
Found 1344 images belonging to 5 classes.


In [6]:
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [9]:
# CREATE THE MODEL ARCHITECTURE

# Exclude the last 5 layers of the above model.
# This will include all layers up to and including global_average_pooling2d_1
x = mobile.layers[-6].output

# Create a new dense layer for predictions
# 7 corresponds to the number of classes
x = Dropout(0.25)(x)
predictions = Dense(5, activation='softmax')(x)

# inputs=mobile.input selects the input layer, outputs=predictions refers to the
# dense layer we created above.

model = Model(inputs=mobile.input, outputs=predictions)

In [13]:
for layer in model.layers[:-6]:
    layer.trainable = False

In [14]:
from tensorflow.keras.optimizers import Adam


model.compile(optimizer=Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [17]:
from tensorflow.keras.callbacks import  ModelCheckpoint


filepath = "F:/yasmin/roi/new_model_check_mobile.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [18]:
history = model.fit(train_batches, steps_per_epoch = 5052 // 64, 
   validation_data=validation_batches, validation_steps = 1262 // 64, epochs=3, callbacks=callbacks_list, verbose=1)

Epoch 1/3
78/78 [==============================] - ETA: 0s - loss: 1.4950 - accuracy: 0.4743
Epoch 00001: val_accuracy improved from -inf to 0.42845, saving model to F:/yasmin/roi/new_model_check_mobile.h5
78/78 [==============================] - 428s 5s/step - loss: 1.4950 - accuracy: 0.4743 - val_loss: 6.0706 - val_accuracy: 0.4285
Epoch 2/3
78/78 [==============================] - ETA: 0s - loss: 1.2577 - accuracy: 0.5285
Epoch 00002: val_accuracy improved from 0.42845 to 0.43092, saving model to F:/yasmin/roi/new_model_check_mobile.h5
78/78 [==============================] - 432s 6s/step - loss: 1.2577 - accuracy: 0.5285 - val_loss: 4.6730 - val_accuracy: 0.4309
Epoch 3/3
78/78 [==============================] - ETA: 0s - loss: 1.2034 - accuracy: 0.5307
Epoch 00003: val_accuracy improved from 0.43092 to 0.43257, saving model to F:/yasmin/roi/new_model_check_mobile.h5
78/78 [==============================] - 365s 5s/step - loss: 1.2034 - accuracy: 0.5307 - val_loss: 4.3810 - val_acc

In [19]:
model.save('F:/yasmin/roi/model_roi_save_mobile.h5')

In [25]:
# Make last block of the conv_base trainable:
for layer in model.layers[:-23]:
    layer.trainable = False

In [26]:
model.compile(optimizer=Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [27]:

filepath = "F:/yasmin/roi/new_model_check_mobile2.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [28]:
history = model.fit(train_batches, steps_per_epoch = 5052 // 64, 
   validation_data=validation_batches, validation_steps = 1262 // 64, epochs=10, callbacks=callbacks_list, verbose=1)

Epoch 1/10
78/78 [==============================] - ETA: 0s - loss: 1.1031 - accuracy: 0.5613
Epoch 00001: val_accuracy improved from -inf to 0.46053, saving model to F:/yasmin/roi/new_model_check_mobile2.h5
78/78 [==============================] - 378s 5s/step - loss: 1.1031 - accuracy: 0.5613 - val_loss: 1.7759 - val_accuracy: 0.4605
Epoch 2/10
78/78 [==============================] - ETA: 0s - loss: 1.0751 - accuracy: 0.5756
Epoch 00002: val_accuracy improved from 0.46053 to 0.49671, saving model to F:/yasmin/roi/new_model_check_mobile2.h5
78/78 [==============================] - 338s 4s/step - loss: 1.0751 - accuracy: 0.5756 - val_loss: 1.3277 - val_accuracy: 0.4967
Epoch 3/10
78/78 [==============================] - ETA: 0s - loss: 1.0618 - accuracy: 0.5816
Epoch 00003: val_accuracy did not improve from 0.49671
78/78 [==============================] - 331s 4s/step - loss: 1.0618 - accuracy: 0.5816 - val_loss: 1.2643 - val_accuracy: 0.4704
Epoch 4/10
78/78 [========================

In [29]:
model.save('F:/yasmin/roi/model_roi_save_mobile2.h5')

In [30]:
# Make last block of the conv_base trainable:

for layer in model.layers[:]:
    layer.trainable = True

In [31]:
model.compile(optimizer=Adam(lr=.00001), loss='categorical_crossentropy', metrics=['accuracy'])


In [32]:
filepath = "F:/yasmin/roi/new_model_check_mobile3.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(train_batches, steps_per_epoch = 5052 // 64, 
   validation_data=validation_batches, validation_steps = 1262 // 64, epochs=37, callbacks=callbacks_list, verbose=1)

Epoch 1/37
78/78 [==============================] - ETA: 0s - loss: 1.2669 - accuracy: 0.5200 
Epoch 00001: val_accuracy improved from -inf to 0.49178, saving model to F:/yasmin/roi/new_model_check_mobile3.h5
78/78 [==============================] - 2523s 32s/step - loss: 1.2669 - accuracy: 0.5200 - val_loss: 1.9711 - val_accuracy: 0.4918
Epoch 2/37
78/78 [==============================] - ETA: 0s - loss: 1.0513 - accuracy: 0.5972 
Epoch 00002: val_accuracy did not improve from 0.49178
78/78 [==============================] - 1773s 23s/step - loss: 1.0513 - accuracy: 0.5972 - val_loss: 1.6197 - val_accuracy: 0.4581
Epoch 3/37
78/78 [==============================] - ETA: 0s - loss: 0.9608 - accuracy: 0.6371 
Epoch 00003: val_accuracy improved from 0.49178 to 0.56168, saving model to F:/yasmin/roi/new_model_check_mobile3.h5
78/78 [==============================] - 1680s 22s/step - loss: 0.9608 - accuracy: 0.6371 - val_loss: 1.1416 - val_accuracy: 0.5617
Epoch 4/37
78/78 [===============

78/78 [==============================] - 1620s 21s/step - loss: 0.5951 - accuracy: 0.7658 - val_loss: 0.6229 - val_accuracy: 0.7771
Epoch 28/37
78/78 [==============================] - ETA: 0s - loss: 0.5963 - accuracy: 0.7799 
Epoch 00028: val_accuracy did not improve from 0.77714
78/78 [==============================] - 1630s 21s/step - loss: 0.5963 - accuracy: 0.7799 - val_loss: 0.6573 - val_accuracy: 0.7508
Epoch 29/37
78/78 [==============================] - ETA: 0s - loss: 0.5798 - accuracy: 0.7811 
Epoch 00029: val_accuracy did not improve from 0.77714
78/78 [==============================] - 1622s 21s/step - loss: 0.5798 - accuracy: 0.7811 - val_loss: 0.6501 - val_accuracy: 0.7558
Epoch 30/37
78/78 [==============================] - ETA: 0s - loss: 0.5824 - accuracy: 0.7807 
Epoch 00030: val_accuracy did not improve from 0.77714
78/78 [==============================] - 1648s 21s/step - loss: 0.5824 - accuracy: 0.7807 - val_loss: 0.6419 - val_accuracy: 0.7640
Epoch 31/37
78/78 [

In [ ]:
model.save('F:/yasmin/roi/model_roi_save_mobile3.h5')